Cellule Variables

In [ ]:
#Métro Nation
#Lat=48.849077
#Lon=2.398671
#Rayon=200

#7 place de la Flourie, 35400 Saint Malo
#Lat = 48.620613
#Lon = -2.009287
#Rayon = 400

#Gare de Becon
Lat = 48.906006
Lon = 2.268197
Rayon = 500

#Bérault métro
#Lat = 48.845392
#Lon = 2.429531
#Rayon = 700

#Vincennes
#Lat = 48.844779
#Lon = 2.437233
#Rayon = 700

#Saint mandé métro
#Lat = 48.846334
#Lon = 2.419169
#Rayon = 200

frais = 0.05

Structure

In [ ]:

setClass(
    Class = "Vente_Lot",
    representation = representation(
        adresse = "character",
        date = "Date",
        prix = "numeric",
        ventes = "vector"
    ),
    prototype = prototype(
        adresse = "Au pif",
        date = as.Date("2020-09-16", "%Y-%m-%d"),
        prix = 0,
        ventes = list(0)
    )
)

setClass(
    Class = "Vente_Unitaire",
    representation = representation(
        surface = "numeric",
        type = "character",
        date = "Date",
        adresse = "character",
        prix = "numeric",
        pieces = "numeric"
    ),
    prototype = prototype(
        surface = 0,
        type = "test",
        date = as.Date("2020-09-16", "%Y-%m-%d"),
        adresse = "Au pif",
        prix = 0,
        pieces = 0
    )
)

creation_lot <- function(premiere_vente) {
    Nouveau_Lot = new(
        Class = "Vente_Lot",
        adresse = premiere_vente@adresse,
        date = premiere_vente@date,
        prix = premiere_vente@prix,
        ventes = list(premiere_vente)
    )
    return(Nouveau_Lot)
}

ajout_vente_a_lot <- function(Vente_Lot, Vente_Unitaire) {
    Vente_Lot@ventes = rbind(Vente_Lot@ventes, list(Vente_Unitaire))
    return(Vente_Lot)
}

appartient_vente_a_lot <- function(Vente_Lot, Vente_Unitaire) {
    if (Vente_Lot@adresse == Vente_Unitaire@adresse &
        Vente_Lot@date == Vente_Unitaire@date) {
        retour = TRUE
    } else{
        retour = FALSE
    }
    return(retour)
}

correction_prix_moy <- function(Vente_Lot) {
    for (i in 1:length(Vente_Lot@ventes)) {
        Vente_Lot@ventes[[i]]@prix = Vente_Lot@prix * Vente_Lot@ventes[[i]]@surface /
            surface_lot(Vente_Lot)
    }
    return(Vente_Lot)
}

surface_lot <- function(Vente_Lot) {
    surface_totale = 0
    for (i in 1:length(Vente_Lot@ventes)) {
        surface_totale = surface_totale + Vente_Lot@ventes[[i]]@surface
    }
    return(surface_totale)
}

Main

In [4]:
library(RJSONIO)
library(RCurl)
library("lubridate")
library(jsonlite)


#La variable lien permet de construire la requete sur l'API DVF.
Lien = paste("http://api.cquest.org/dvf?lat=",
             Lat,
             "&lon=",
             Lon,
             "&dist=",
             Rayon,
             sep = "")
download.file(Lien, destfile = "dvf.json")

jdata <- fromJSON("dvf.json")
data <- as.data.frame(jdata)
write.csv(data$features.properties,
          file = "dvf.csv",
          row.names = TRUE)
donnees  	<- read.csv('dvf.csv', sep = ',', header = TRUE)

n_ventes = nrow(donnees)
donnees$date_mutation = as.Date(donnees$date_mutation, "%Y-%m-%d")

premiere_adresse = paste(donnees[1, 6], donnees[1, 7], donnees[1, 9], donnees[1, 10], donnees[1, 11], sep =
                             " ")
premiere_vente = new(
    Class = "Vente_Unitaire",
    surface = donnees[1, 21],
    type = as.character(donnees[1, 20]),
    date = donnees[1, 3],
    adresse = premiere_adresse,
    prix = donnees[1, 5]
)
premier_lot = creation_lot(premiere_vente)
donnees_structurees = list(premier_lot)

for (i in 2:n_ventes) {
    adresse_nouvelle = paste(donnees[i, 6], donnees[i, 7], donnees[i, 9], donnees[i, 10], donnees[i, 11], sep =
                                 " ")
    nouvelle_vente = new(
        Class = "Vente_Unitaire",
        surface = donnees[i, 21],
        type = as.character(donnees[i, 20]),
        date = donnees[i, 3],
        adresse = adresse_nouvelle,
        prix = donnees[i, 5],
        pieces = donnees[i, 22]
    )
    vente_liee = FALSE
    for (j in 1:length(donnees_structurees)) {
        if (appartient_vente_a_lot(donnees_structurees[[j]], nouvelle_vente)) {
            donnees_structurees[[j]] = ajout_vente_a_lot(donnees_structurees[[j]], nouvelle_vente)
            vente_liee = TRUE
        }
    }
    if (vente_liee == FALSE) {
        nouveau_lot = creation_lot(nouvelle_vente)
        donnees_structurees = rbind(donnees_structurees, list(nouveau_lot))
    }
}

for (i in 1:length(donnees_structurees)) {
    donnees_structurees[[i]] = correction_prix_moy(donnees_structurees[[i]])
}

donnees_retraitees <- data.frame(matrix(ncol = 7, nrow = 1))
colnames(donnees_retraitees) <- c(
    "date_mutation",
    "adresse",
    "nombre_pieces_principales",
    "valeur_fonciere",
    "type_local",
    "surface_relle_bati",
    "prix_m2"
)
donnees_retraitees$date_mutation[1] = year(as.Date(as.character(donnees_structurees[[1]]@date), "%Y-%m-%d"))
donnees_retraitees$adresse[1] = donnees_structurees[[1]]@adresse
donnees_retraitees$nombre_pieces_principales[1] = donnees_structurees[[1]]@ventes[[1]]@pieces
donnees_retraitees$valeur_fonciere[1] = donnees_structurees[[1]]@ventes[[1]]@prix
donnees_retraitees$type_local[1] = donnees_structurees[[1]]@ventes[[1]]@type
donnees_retraitees$surface_relle_bati[1] = donnees_structurees[[1]]@ventes[[1]]@surface
donnees_retraitees$prix_m2[1] = donnees_retraitees$valeur_fonciere[1] /
    donnees_retraitees$surface_relle_bati[1]

for (i in 2:length(donnees_structurees)) {
    for (j in 1:length(donnees_structurees[[i]])) {
        if (!(is.na(donnees_structurees[[i]]@ventes[[j]]@type))) {
            if (donnees_structurees[[i]]@ventes[[j]]@type != "Dépendance" &
                donnees_structurees[[i]]@ventes[[j]]@type != "Local industriel. commercial ou assimilé")
                donnees_retraitees = rbind(
                    donnees_retraitees,
                    c(
                        year(as.Date(
                            as.character(donnees_structurees[[i]]@date),
                            "%Y-%m-%d"
                        )),
                        donnees_structurees[[i]]@adresse,
                        donnees_structurees[[i]]@ventes[[j]]@pieces,
                        donnees_structurees[[i]]@ventes[[j]]@prix,
                        donnees_structurees[[i]]@ventes[[j]]@type,
                        donnees_structurees[[i]]@ventes[[j]]@surface,
                        donnees_structurees[[i]]@ventes[[j]]@prix / donnees_structurees[[i]]@ventes[[j]]@surface
                    )
                )
        }
    }
}
write.csv(donnees_retraitees,
          file = "donnees_retraitees.csv",
          row.names = TRUE)

Types    = sort(unique(donnees_retraitees$type_local))
Annees   = sort(unique(donnees_retraitees$date_mutation))
Pieces = sort(unique(donnees_retraitees$nombre_pieces_principales))

n_types  = length(Types)
n_annees = length(Annees)
n_pieces = length(Pieces)

#Construction de la df des résultats

#Alimentation de la df
#Pour chaque genre de logement (appart/dép/local), on calcule le prix moyen par année et par Type de logement (nombre de pieces).
#Un type de logement => une courbe de prix moyen.
#Pour chaque donnée, on extrait de la base brute les données qui correspondent au genre/Année/nb de pièces via subset
#Ensuite soit on initialise la première ligne 'pligne = True', soit on ajoute les données à la DF de résultats
pligne = TRUE
for (i in Types) {
    for (j in Annees) {
        for (k in Pieces) {
            sub_donnees_retraitees = subset(
                donnees_retraitees,
                type_local == as.character(i) &
                    date_mutation == as.character(j) &
                    nombre_pieces_principales == as.character(k),
                select = "prix_m2"
            )
            n_ventes = nrow(sub_donnees_retraitees)
            prix_m2 = mean(as.numeric(sub_donnees_retraitees$prix_m2))
            if (pligne == TRUE) {
                df_prix_moy <- data.frame(matrix(ncol = 5, nrow = 1))
                colnames(df_prix_moy) <-
                    c("Type",
                      "Année",
                      "Pièces",
                      "Prix_moyen",
                      "Nombre_ventes")
                df_prix_moy[1,] = c(i, j, k, prix_m2, n)
                pligne = FALSE
            } else{
                df_prix_moy = rbind(df_prix_moy, c(i, j, k, prix_m2, n_ventes))
            }
        }
    }
}

#Construction de la df des résultats
pligne = TRUE
#Alimentation de la df
#Pour chaque genre de logement (appart/dép/local), on calcule le prix moyen par année.
#Pour chaque donnée, on extrait de la base brute les données qui correspondent au genre/Année via subset
#Ensuite soit on initialise la première ligne 'pligne = True', soit on ajoute les données à la DF de résultats
for (i in Types) {
    for (j in Annees) {
        sub_donnees_retraitees = subset(
            donnees_retraitees,
            type_local == as.character(i) &
                date_mutation == as.character(j),
            select = "prix_m2"
        )
        n = nrow(sub_donnees_retraitees)
        prix_m2 = mean(as.numeric(sub_donnees_retraitees$prix_m2))
        x = c(i, j, prix_m2, n)
        if (pligne == TRUE) {
            df_prix_moy2 <- data.frame(matrix(ncol = 4, nrow = 1))
            colnames(df_prix_moy2) <-
                c("Type", "Année", "Prix_moyen", "Nombre_ventes")
            df_prix_moy2[1,] = c(i, j, prix_m2, n)
            pligne = FALSE
        } else{
            df_prix_moy2 = rbind(df_prix_moy2, x)
        }
    }
}

#Construction des matrices de résulats
#Création des 4 matrices de résultats
#Deux matrices (a et b) pour les prix et le nombre de ventes par nombre de pièces
#Deux matrices (c et d) pour les prix et le nombre de ventes de manière indiférentiée du nombre de pièces
a = matrix (rep(0, n_annees * n_pieces), n_annees, n_pieces)
rownames(a) = Annees
colnames(a) = Pieces

b = matrix (rep(0, n_annees * n_pieces), n_annees, n_pieces)
rownames(b) = Annees
colnames(b) = Pieces

c = matrix (rep(0, n_annees * 1), n_annees, 1)
rownames(c) = Annees
colnames(c) = c("Global")

d = matrix (rep(0, n_annees * 1), n_annees, 1)
rownames(d) = Annees
colnames(d) = c("Global")

#Ouverture du pdf
pdf(file = "multipass.pdf", onefile = TRUE)

#Première boucle, pour chaque type de logement, pour chaque nombre de pièces et pour chaque année, affiche la courbe du nombre de ventes et du prix moyen.
#Les variables Prix_m2_an_max et Nb_ventes_max servent à dimentionner la taille des fenetres.

for (i in Types) {
    x = 1
    Prix_m2_an_max = 0
    Nb_ventes_max = 0
    for (j in Annees) {
        y = 1
        for (k in Pieces) {
            Prix_m2_an = as.numeric(
                subset(
                    df_prix_moy,
                    Type == as.character(i) &
                        Année == as.character(j) &
                        Pièces == as.character(k),
                    select = c("Prix_moyen")
                )[[1]]
            )
            a[x, y] = Prix_m2_an
            if (is.finite(Prix_m2_an) & Prix_m2_an > Prix_m2_an_max) {
                Prix_m2_an_max = Prix_m2_an
            }
            Nb_ventes = as.numeric(
                subset(
                    df_prix_moy,
                    Type == as.character(i) &
                        Année == as.character(j) &
                        Pièces == as.character(k),
                    select = c("Nombre_ventes")
                )[[1]]
            )
            b[x, y] = Nb_ventes
            if (is.finite(Nb_ventes) & Nb_ventes > Nb_ventes_max) {
                Nb_ventes_max = Nb_ventes
            }
            y = y + 1
        }
        x = x + 1
    }
    print(a)
    matplot(
        rownames(a),
        a,
        type = 'l',
        xlab = 'Année',
        ylab = paste('Prix au m² - ', i),
        ylim = c(0, max(10, Prix_m2_an_max * 1.05))
    )
    legend(
        'topleft',
        col = seq_len(ncol(a)),
        legend = colnames(a),
        lwd = 1
    )
    matplot(
        rownames(b),
        b,
        type = 'l',
        xlab = 'Année',
        ylab = paste('Nombre de ventes par an - ', i),
        ylim = c(0, max(10, Nb_ventes_max * 1.05))
    )
    legend(
        'topleft',
        col = seq_len(ncol(b)),
        legend = colnames(b),
        lwd = 1
    )
}

for (i in Types) {
    x = 1
    Prix_m2_an_max = 0
    Nb_ventes_max = 0
    for (j in Annees) {
        Prix_m2_an = as.numeric(subset(
            df_prix_moy2,
            Type == as.character(i) &
                Année == as.character(j),
            select = c("Prix_moyen")
        )[[1]])
        c[x, 1] = Prix_m2_an
        if (is.finite(Prix_m2_an) & Prix_m2_an > Prix_m2_an_max) {
            Prix_m2_an_max = Prix_m2_an
        }
        Nb_ventes = as.numeric(subset(
            df_prix_moy2,
            Type == as.character(i) &
                Année == as.character(j),
            select = c("Nombre_ventes")
        )[[1]])
        d[x, 1] = Nb_ventes
        if (is.finite(Nb_ventes) & Nb_ventes > Nb_ventes_max) {
            Nb_ventes_max = Nb_ventes
        }
        x = x + 1
    }
    print(c)
    matplot(
        rownames(c),
        c,
        type = 'l',
        xlab = 'Année',
        ylab = paste('Prix au m² - ', i),
        ylim = c(0, max(10, Prix_m2_an_max * 1.05))
    )
    matplot(
        rownames(d),
        d,
        type = 'l',
        xlab = 'Année',
        ylab = paste('Nombre de ventes par an - ', i),
        ylim = c(0, max(10, Nb_ventes_max * 1.05))
    )
}

dev.off()

            0        1        2        3        4
2014      NaN 4666.737 4483.702 5710.450      NaN
2015      NaN      NaN 4682.850 5002.371 5561.581
2016      NaN 5961.538 5696.094 5140.008 5094.118
2017 2207.647 6372.331 5765.610 5396.304 3662.311
2018      NaN 6819.012 6050.296 5810.811 6167.081
2019      NaN      NaN 6329.667 6438.471 5732.759
       Global
2014 4886.497
2015 4927.822
2016 5281.068
2017 5455.277
2018 6123.817
2019 6307.126


png 
  2